In [29]:
from tensorflow.keras import Input
from tensorflow.keras import layers, Model

input_img = Input(shape=(28, 28, 1))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
# print(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
# print(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# print(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
# print(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# print(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
# print(encoded.shape)

encoder = Model(input_img, encoded)

In [32]:
import numpy as np
a = np.zeros([32,28,28,1])
a.shape

(32, 28, 28, 1)

In [33]:
encoder.predict(a).shape

(32, 4, 4, 8)